In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set()

In [3]:
titanic_df = pd.read_csv('train.csv')
titanic_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
# summary statistics for numerical variables:

titanic_df.describe()[['Age', 'SibSp', 'Parch', 'Fare']]

,Age,SibSp,Parch,Fare
count,714.000000,891.000000,891.000000,891.000000
mean,29.699118,0.523008,0.381594,32.204208
std,14.526497,1.102743,0.806057,49.693429
min,0.420000,0.000000,0.000000,0.000000
25%,20.125000,0.000000,0.000000,7.910400
50%,28.000000,0.000000,0.000000,14.454200
75%,38.000000,1.000000,0.000000,31.000000
max,80.000000,8.000000,6.000000,512.329200


**Comment:**

In [16]:
# counts of categorical variables:
print(titanic_df.groupby('Survived').count()['PassengerId'])
print()

print(titanic_df.groupby('Pclass').count()['PassengerId'])
print()

print(titanic_df.groupby('Sex').count()['PassengerId'])
print()

print(titanic_df.groupby('Embarked').count()['PassengerId'])
print()

Survived
0    549
1    342
Name: PassengerId, dtype: int64

Pclass
1    216
2    184
3    491
Name: PassengerId, dtype: int64

Sex
female    314
male      577
Name: PassengerId, dtype: int64

Embarked
C    168
Q     77
S    644
Name: PassengerId, dtype: int64



In [23]:
titanic_df.groupby('Cabin').count()['PassengerId']

# extract a new feature: cabin class?

Cabin
A10    1
A14    1
A16    1
A19    1
A20    1
      ..
F33    3
F38    1
F4     2
G6     4
T      1
Name: PassengerId, Length: 147, dtype: int64